# Python For Finance 読書会#13

## Chapter 7. Multifactor Models and Performance Measures を読む

* Date: 2019/12/06(Fri) 19:30 - @Google Meet
* 発表者：2casa

## 自己紹介

* 2casa（つかさ）
  * Twitter : https://twitter.com/moscow_ii
  * Github : https://github.com/2casa
  
* 某Fintech系Startupに転職したものの、最近は非エンジニアな仕事（書類作成）ばかり。

* References
  * Fundamental Factor Models
  * [www.quantopian.com/lectures](https://www.quantopian.com/lectures)
  * [github.com/quantopian/research_public](https://github.com/quantopian/research_public)
  
 
  

## 復習

CAPMの主張：株式の期待収益率は、市場の期待収益率との間に以下の関係が成り立つ

$$
R_{IBM} = R_f + \beta_{IBM}(R_{mkt} - R_f) 
$$


* $R_{IBM}$ : IBMの期待収益率
* $R_{mkt}$ : 市場の期待収益率
* $R_f$ : 無リスク資産金利（RiskFreeRate）
* $\beta_{IBM}$ : IBMの市場リスク尺度


IBMの期待収益率（左辺）が、市場の期待収益率と無リスク金利の差（「市場超過収益率」）という１つのリスク要因（ファクター）によって説明されるので、シングルファクターモデルと呼ばれる。

マルチファクターモデルでは、ファクターの数を複数に拡張する。ファクターの数を複数に拡張すること自体には理論的な合理性はあるものの、そのファクターが何であるか？ということは要求されないため、ファクターの選択はサイエンスというよりもアート依り。

[参考wikipedia:ファーマ＝フレンチの３ファクターモデル](https://ja.wikipedia.org/wiki/%E3%83%95%E3%82%A1%E3%83%BC%E3%83%9E-%E3%83%95%E3%83%AC%E3%83%B3%E3%83%81%E3%81%AE3%E3%83%95%E3%82%A1%E3%82%AF%E3%82%BF%E3%83%BC%E3%83%A2%E3%83%87%E3%83%AB)

Fama-Frenchモデルは、CAPMを拡張することによって、以下のように表現される。

$$ R_i = R_f + \beta_{mkt}(R_{mkt} - R_f) + \beta_{smb}SMB + \beta_{hml}HML + \epsilon$$


右辺第二項までは、CAPMと同じ。第３項以降（SMB, HML）が新たに追加された項。
要素が1つ（MKT）から、3つ（MKT, SMB, HML）に増えたので「マルチファクター」と呼ばれる。

この中で不明なもの
* SMB :  サイズファクター（小型株効果）に対する期待リターン
* HML :  バリューファクター（バリュー株効果）に対する期待リターン
* 各種$\beta$ ：市場ファクター、サイズファクター、バリューファクターに対する、各銘柄の感応度（ベータ）

### モデル構築手順

1. SMB, HML の値を計算する
　　* $R_{mkt}$ は、S&P500ETFのリターンだった。
  * SMBは小型株をロング、大型株をショートしたペーパーポートフォリオを構築し、そのリターンを計測する 。
  * HMLはバリュー株をロング、グロース株をショートする。
    * PBR(株価純資産倍率＝株価/簿価)
    * PBR<1 なら、時価が簿価よりも安い（割安）、PBR>1なら、簿価よりも時価が高い（プレミアムがついてる）
      * HMLは PBRの逆数を取ることで、PBRが大きければ大きいほど割安、小さければ小さいほど割高、という指標に変換している。
2. MKT, SMB, HML （ファクター）を説明変数、$R_i$ を目的変数として回帰分析を行うことでβを推定する。

### Portfolio Construction as an Asset Pricing Model

First we import the relevant libraries.

In [ ]:
import pandas as pd
import numpy as np
from quantopian.pipeline import Pipeline
from quantopian.pipeline.data import Fundamentals
from quantopian.pipeline.factors import CustomFactor, Returns, Latest
from quantopian.pipeline.classifiers import Classifier
from quantopian.pipeline.filters import QTradableStocksUS
from quantopian.research import run_pipeline
import matplotlib.pyplot as plt

In [ ]:
# 4factorで使います.
class Momentum(CustomFactor):
    # will give us the returns from last month
    inputs = [Returns(window_length=20)]
    window_length = 20
    
    def compute(self, today, assets, out, lag_returns):
        out[:] = lag_returns[0]

def make_pipeline():
    # define our fundamental factor pipeline
    pipe = Pipeline()
    
    # 時価総額
    market_cap = Latest([Fundamentals.market_cap])
    # 簿価時価比率（Quantopianでは、時価簿価比率（PBR）として収録されているので、逆数をとります）
    book_to_price = 1/Latest([Fundamentals.pb_ratio])
    # モメンタム（4factor）
    momentum = Momentum()
    # R_i として日次リターンをとります
    returns = Returns(window_length=2)
    
    # 時価総額、簿価時価比率でランキングする
    market_cap_rank = market_cap.rank(mask=QTradableStocksUS())
    book_to_price_rank = book_to_price.rank(mask=QTradableStocksUS())
    
    # モメンタムでランキングする（４factors)
    momentum_rank = momentum.rank(mask=QTradableStocksUS())
    
    # Quantopian仕様
    #biggest = market_cap_rank.top(1000)
    #smallest = market_cap_rank.bottom(1000)
    #highpb = book_to_price_rank.top(1000)
    #lowpb = book_to_price_rank.bottom(1000)
    #top = momentum_rank.top(1000)
    #bottom = momentum_rank.bottom(1000)
    
    # PythonForFinance仕様()
    biggest = market_cap.percentile_between(50,100)
    smallest = market_cap.percentile_between(0,50)
    highpb = book_to_price.percentile_between(70,100)
    midpb = book_to_price.percentile_between(30,70)
    lowpb = book_to_price.percentile_between(0,30)
    top = momentum.percentile_between(50,100)
    bottom = momentum.percentile_between(0,50)
    
    
    # Define our universe, screening out anything that isn't in the top or bottom
    universe = QTradableStocksUS() & (biggest | smallest | highpb | lowpb | top | bottom)
    
    pipe = Pipeline(
        columns = {
            'market_cap':market_cap,
            'book_to_price':book_to_price,
            'momentum':momentum,
            'Returns':returns,
            'market_cap_rank':market_cap_rank,
            'book_to_price_rank':book_to_price_rank,
            'momentum_rank':momentum_rank,
            'biggest':biggest,
            'smallest':smallest,
            'highpb':highpb,
            'midpb':midpb,
            'lowpb':lowpb,
            'top':top,
            'bottom':bottom
        },
        screen=universe
    )
    
    return pipe

# Initializing the pipe
pipe = make_pipeline()

# Now let's start the pipeline
start_date, end_date = '2016-01-01', '2016-12-31' 

results = run_pipeline(pipe, start_date, end_date)

results.head()

Now we can go through the data and build the factor portfolios we want

In [ ]:
#　ポートフォリオのリターンを計測、、、してるのかと思いきや、リターンの平均を取ることで代用しているっぽい。
R_biggest = results[results.biggest]['Returns'].groupby(level=0).mean()
R_smallest = results[results.smallest]['Returns'].groupby(level=0).mean()

R_highpb = results[results.highpb]['Returns'].groupby(level=0).mean()
R_lowpb = results[results.lowpb]['Returns'].groupby(level=0).mean()

R_top = results[results.top]['Returns'].groupby(level=0).mean()
R_bottom = results[results.bottom]['Returns'].groupby(level=0).mean()

# risk-free proxy
R_F = get_pricing('BIL', fields='price', start_date=start_date, end_date=end_date).pct_change()[1:]

# find it's beta against market
M = get_pricing('SPY', start_date=start_date, end_date=end_date, fields='price').pct_change()[1:]

# Defining our final factors
EXMRKT = M - R_F
SMB = R_smallest - R_biggest # small minus big
HML = R_highpb - R_lowpb # high minus low
MOM = R_top - R_bottom # momentum

Now that we've constructed our portfolios, let's look at our performance if we were to hold each one.

In [ ]:
plt.plot(SMB.index, SMB.values)
plt.ylabel('Daily Percent Return')
plt.legend(['SMB Portfolio Returns']);

In [ ]:
plt.plot(HML.index, HML.values)
plt.ylabel('Daily Percent Return')
plt.legend(['HML Portfolio Returns']);

In [ ]:
plt.plot(MOM.index, MOM.values)
plt.ylabel('Daily Percent Return')
plt.legend(['MOM Portfolio Returns']);

Now, as we did in the CAPM lecture, we'll calculate the risk premia on each of these factors using the Fama-Macbeth regressions.

Fama-Macbeth回帰を行うことで、それぞれのファクターに対する感応度を計算します。



In [ ]:
import itertools
import statsmodels.api as sm
from statsmodels import regression,stats
import scipy

Our asset returns data is asset and date specific, whereas our factor portfolio returns are only date specific. Therefore, we'll need to spread each day's portfolio return across all the assets for which we have data for on that day.

個別銘柄のリターン情報というのは、$R_{i,t}$、つまり、「銘柄コード」と「時刻」の２つの添字によって構成されます。（２次元）
一方で、SMB、HMLのポートフォリオリターンは、$SMB_{t}$、つまり、「時刻」という１つの添字によって構成されます。（１次元）
したがって各営業日ごとに、ファクターポートフォリオのデータを個別銘柄に対して拡張する必要があります。



In [ ]:
data = results[['Returns']].set_index(results.index)
asset_list_sizes = [group[1].size for group in data.groupby(level=0)]

# Spreading the factor portfolio data across all assets for each day
SMB_column = [[SMB.loc[group[0]]] * size for group, size \
              in zip(data.groupby(level=0), asset_list_sizes)]
data['SMB'] = list(itertools.chain(*SMB_column))

HML_column = [[HML.loc[group[0]]] * size for group, size \
              in zip(data.groupby(level=0), asset_list_sizes)]
data['HML'] = list(itertools.chain(*HML_column))

MOM_column = [[MOM.loc[group[0]]] * size for group, size \
              in zip(data.groupby(level=0), asset_list_sizes)]
data['MOM'] = list(itertools.chain(*MOM_column))

EXMRKT_column = [[EXMRKT.loc[group[0]]]*size if group[0] in EXMRKT.index else [None]*size \
                 for group, size in zip(data.groupby(level=0), asset_list_sizes)]

data['EXMRKT'] = list(itertools.chain(*EXMRKT_column))

data = sm.add_constant(data.dropna())

# Our list of assets from pipeline
assets = data.index.levels[1].unique()

# gathering our data to be asset-specific
Y = [data.xs(asset, level=1)['Returns'] for asset in assets] 
X = [data.xs(asset, level=1)[['EXMRKT','SMB', 'HML', 'MOM', 'const']] for asset in assets]

# First regression step: estimating the betas
reg_results = [regression.linear_model.OLS(y, x).fit().params \
               for y, x in zip(Y, X) if not(x.empty or y.empty)]
indices = [asset for y, x, asset in zip(Y, X, assets) if not(x.empty or y.empty)]

betas = pd.DataFrame(reg_results, index=indices)
betas = sm.add_constant(betas.drop('const', axis=1))

R = data['Returns'].mean(axis=0, level=1)

# Second regression step: estimating the risk premia
risk_free_rate = np.mean(R_F)

final_results = regression.linear_model.OLS(R - risk_free_rate, betas).fit()

final_results.summary()

#### Returns Prediction
As discussed in the CAPM lecture, factor modeling can be used to predict future returns based on current fundamental factors. As well, it could be used to determine when an asset may be mispriced in order to arbitrage the difference, as shown in the CAPM lecture.

Modeling future returns is accomplished by offsetting the returns in the regression, so that rather than predict for current returns, you are predicting for future returns. Once you have a predictive model, the most canonical way to create a strategy is to attempt a long-short equity approach.